In [5]:
import pandas as pd

df = pd.read_csv("glass-data.csv", sep="\t")
df.columns = df.columns.str.strip()  

# Đổi tên cột để dễ thực hiện
df.columns = [
    "Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Type"
]

# a. Kích thước dữ liệu
print("a. Kích thước dữ liệu:", df.shape)
print("   Số hàng (mẫu):", df.shape[0])
print("   Số cột (thuộc tính):", df.shape[1])

# b. Kiểu dữ liệu
print("\nb. Kiểu dữ liệu của các thuộc tính:")
print(df.dtypes)

# c. Số lượng từng giá trị nhãn (cột Type)
print("\nc. Số lượng thực thể của từng giá trị nhãn:")
print(df["Type"].value_counts())

# d. Giá trị min, max, mean
print("\nd. Giá trị thống kê của các cột số thực:")
print(df.describe().loc[["min", "max", "mean"]])



a. Kích thước dữ liệu: (214, 11)
   Số hàng (mẫu): 214
   Số cột (thuộc tính): 11

b. Kiểu dữ liệu của các thuộc tính:
Id        int64
RI      float64
Na      float64
Mg      float64
Al      float64
Si      float64
K       float64
Ca      float64
Ba      float64
Fe      float64
Type      int64
dtype: object

c. Số lượng thực thể của từng giá trị nhãn:
Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64

d. Giá trị thống kê của các cột số thực:
         Id        RI        Na        Mg        Al         Si         K  \
min     1.0  1.511150  10.73000  0.000000  0.290000  69.810000  0.000000   
max   214.0  1.533930  17.38000  4.490000  3.500000  75.410000  6.210000   
mean  107.5  1.518365  13.40785  2.684533  1.444907  72.650935  0.497056   

             Ca        Ba        Fe      Type  
min    5.430000  0.000000  0.000000  1.000000  
max   16.190000  3.150000  0.510000  7.000000  
mean   8.956963  0.175047  0.057009  2.780374  


In [5]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# === 1. Load data ===
df = pd.read_csv("glass-data.csv", sep="\t")

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# === 2. Define CV & scorer ===
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
f1 = make_scorer(f1_score, average='macro')

# === 3. Models & params ===
models = {
    "KNN": (
        Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier())]),
        {"knn__n_neighbors": [3, 5, 7, 9],
         "knn__weights": ["uniform", "distance"]}
    ),
    
    "Random Forest": (
        RandomForestClassifier(random_state=42),
        {"n_estimators": [100, 200, 300],
         "max_depth": [None, 5, 10, 20],
         "min_samples_split": [2, 5]}
    ),
    
    "SVM (RBF)": (
        Pipeline([("scaler", StandardScaler()), ("svm", SVC())]),
        {"svm__C": [0.1, 1, 10],
         "svm__gamma": ["scale", "auto"],
         "svm__kernel": ["rbf"]}
    )
}

# === 4. Training & saving results ===
rows = []

for name, (model, params) in models.items():
    grid = GridSearchCV(model, params, cv=kfold, scoring=f1, n_jobs=-1)
    grid.fit(X, y)

    rows.append([
        name,
        round(grid.best_score_, 4),
        grid.best_params_
    ])

results = pd.DataFrame(rows, columns=["Model", "F1 Score (Macro)", "Best Hyperparameters"])

# === 5. Display better formatted output ===
print("\n KẾT QUẢ SO SÁNH MÔ HÌNH\n")
for idx, row in results.iterrows():
    print(f"🔹 Model: {row['Model']}")
    print(f"   F1 Score (Macro): {row['F1 Score (Macro)']}")
    print(f"   Best Params:")
    for k, v in row["Best Hyperparameters"].items():
        print(f"      {k}: {v}")
    print("-" * 50)



 KẾT QUẢ SO SÁNH MÔ HÌNH

🔹 Model: KNN
   F1 Score (Macro): 0.7873
   Best Params:
      knn__n_neighbors: 3
      knn__weights: distance
--------------------------------------------------
🔹 Model: Random Forest
   F1 Score (Macro): 0.9765
   Best Params:
      max_depth: None
      min_samples_split: 2
      n_estimators: 200
--------------------------------------------------
🔹 Model: SVM (RBF)
   F1 Score (Macro): 0.8221
   Best Params:
      svm__C: 10
      svm__gamma: scale
      svm__kernel: rbf
--------------------------------------------------
